In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import os #deal with directories and can work on all OS

In [ ]:
## Path definitions
PROJECT_ROOT = os.path.join(os.getcwd(), '..') #Main file (in this case "AI_Analysis_and_Chatbot")
EXCEL_FILE_PATH = os.path.join(PROJECT_ROOT, 'Data', 'amazon_product_reviews.xlsx') #gets path with file name and folder

In [ ]:
## Data cleaning functions

# Cleaning numbers
def clean_num (series): #remove symbols from numbers and and convert to float
    if series.dtype == 'object': #when reading it appears as text (so we dont take numbers without symbols)
        return (series.astype(str) #cast to string for consistency
                      .str.replace('â‚¹', '', regex=False) #delete symbol (replace with empty string)
                      .str.replace(',', '', regex=False) #so they can be dealt with as floats/ints not strings
                      .str.replace('%', '', regex=False)
                      .str.strip() #remove spaces before/after string
                      .astype(float, errors='coerce')) # Convert to float, turns errors (like empty strings) to NaN
    return series

# Cleaning text
def clean_txt(series):
    if series.dtype == 'object':
        return (series.astype(str)
                .str.replace(r'[ã€\s?\[?\u200b\ufeff\u00a0\u201d\u201c\u2013\u2022âœ“â€¢â˜…ã€‘]', '', regex=True) #remove symbols
                .str.strip()
                .str.replace(r'\s+', ' ', regex=True) #replace multispace with one space
                )
    return series

In [8]:
## Data Reading

try:
    df = pd.read_excel(EXCEL_FILE_PATH, engine='openpyxl') #openpyxl is modern engine for .xlsx format
    print(f"Successfully loaded {len(df)} records.") #prove file was read correctly and how many records we have
except FileNotFoundError:
    print(f"ERROR: File not in {EXCEL_FILE_PATH}")
    exit()


Successfully loaded 1465 records.


In [ ]:
## Data Cleaning